## Sample Data for Enterprise Question-Answering System

### 1. Sales Data

Sales data will consist of:
- Sales records (.csv) downloaded from Kaggle
- Sales conversations for each record, generated by an LLM

We first download the .csv file from kaggle (https://www.kaggle.com/datasets/kyanyoga/sample-sales-data) which contains structured sales records

In [1]:
#Read the CSV
import os
import pandas as pd
sales_data_df = pd.read_csv('../data-samples/sales_data_sample.csv', encoding='iso-8859-1')
#This dataset has 2k+ rows
#Lets shorten this dataset to speed up things
sales_data_df = sales_data_df.groupby('PRODUCTLINE').apply(lambda x: x.sample(n=20, random_state=42)).reset_index(drop=True)
sales_data_df

/tmp/ipykernel_89142/1910503490.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sales_data_df = sales_data_df.groupby('PRODUCTLINE').apply(lambda x: x.sample(n=20, random_state=42)).reset_index(drop=True)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10212,33,100.00,15,4180.44,1/16/2004 0:00,Shipped,1,1,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium
1,10412,60,100.00,9,11887.80,5/3/2005 0:00,Shipped,2,5,2005,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Large
2,10363,21,100.00,15,2447.76,1/6/2005 0:00,Shipped,1,1,2005,...,"Software Engineering Center, SEC Oy",NaN,Espoo,NaN,FIN-02271,Finland,EMEA,Suominen,Kalle,Small
3,10272,25,100.00,5,3734.00,7/20/2004 0:00,Shipped,3,7,2004,...,7586 Pompton St.,NaN,Allentown,PA,70267,USA,NaN,Yu,Kyung,Medium
4,10281,44,100.00,9,7020.64,8/19/2004 0:00,Shipped,3,8,2004,...,7586 Pompton St.,NaN,Allentown,PA,70267,USA,NaN,Yu,Kyung,Large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,10420,45,26.88,1,1209.60,5/29/2005 0:00,In Process,2,5,2005,...,"Monitor Money Building, 815 Pacific Hwy",Level 6,Chatswood,NSW,2067,Australia,APAC,Huxley,Adrian,Small
136,10149,30,58.22,3,1746.60,9/12/2003 0:00,Shipped,3,9,2003,...,2793 Furth Circle,NaN,Brisbane,CA,94217,USA,NaN,Taylor,Sue,Small
137,10119,27,99.52,8,2687.04,4/28/2003 0:00,Shipped,2,4,2003,...,Geislweg 14,NaN,Salzburg,NaN,5020,Austria,EMEA,Pipps,Georg,Small
138,10288,31,100.00,2,3822.92,9/1/2004 0:00,Shipped,3,9,2004,...,Village Close - 106 Linden Road Sandown,2nd Floor,Singapore,NaN,69045,Singapore,APAC,Victorino,Wendy,Medium


The csv file has several columns about the sale that takes place. In order to create some unstructured data, we use Llama-3.1 to generate some fake converations between a customer and a salesperson based on each row of the sales data. This will be embedded and put into a vector store for retrieval during query time.

We use utility code to generate the sales conversations:

In [ ]:
from generate_sales_conversation import generate_sales_data

sales_data_df = generate_sales_data(sales_data_df)

In [3]:
sales_data_df[['PRODUCTLINE', 'ORDERNUMBER', 'CUSTOMERNAME', 'CONVERSATION']]

,PRODUCTLINE,ORDERNUMBER,CUSTOMERNAME,CONVERSATION
0,Classic Cars,10212,Euro Shopping Channel,"{\n ""Customer"": ""Hi, I'm from Euro Shopping C..."
1,Classic Cars,10412,Euro Shopping Channel,"{\n ""Customer"": ""Hello, I'm Diego Freyre from..."
2,Classic Cars,10363,Suominen Souveniers,"{\n ""Customer"": ""Hi, I'm Kalle from Suominen ..."
3,Classic Cars,10272,Diecast Classics Inc.,"{\n ""Customer"": ""Hi, I'm Kyung Yu from Diecas..."
4,Classic Cars,10281,Diecast Classics Inc.,"{\n ""Customer"": ""Hi, I'm from Diecast Classic..."
...,...,...,...,...
135,Vintage Cars,10420,Souveniers And Things Co.,"{\n ""Customer"": ""Hi, I'm Adrian Huxley from S..."
136,Vintage Cars,10149,Signal Collectibles Ltd.,"{\n ""Customer"": ""Hi, I'm Sue Taylor from Sign..."
137,Vintage Cars,10119,Salzburg Collectables,"{\n ""Customer"": ""Hi, I'm interested in purcha..."
138,Vintage Cars,10288,Handji Gifts& Co,"{\n ""Customer"": ""Hi, I'm Victorino from Handj..."


In [5]:
#Check one of the conversations
print(sales_data_df.iloc[0]['CONVERSATION'])

{
  "Customer": "Hi, I'm from Euro Shopping Channel, a TV shopping channel. We're interested in buying classic cars. Can you show me some options?",
  "Salesman": "Welcome to our classic car dealership, Euro Shopping Channel. We have a wide range of classic cars from various brands. What's your budget for this purchase?",
  "Customer": "We're looking for something within our budget of 1000 dollars. Do you have any classic cars that fit within this price range?",
  "Salesman": "We have a few options that fit within your budget. This 1960s Mustang is one of them. It's a rare model and comes with a lot of features. Would you like to take a look?",
  "Customer": "Yes, I'd love to see it. Can you tell me more about the car's condition and history?",
  "Salesman": "This car has been well-maintained and has a clean title. It's a one-owner vehicle and has been garage-kept its entire life. We also have documentation on its history and any previous repairs.",
  "Customer": "That sounds great. Wh

In [6]:
#Save to csv
sales_data_df.to_csv('../data-samples/sales_data_sample_clean.csv', index=False)

### 2. Ticketing Data

Ticketing Data will consist of:
- Customer Support tickets (.csv) downloaded from kaggle

We first download the .csv file from kaggle (https://www.kaggle.com/datasets/suraj520/customer-support-ticket-dataset/code) and load it, which contains support ticket data:

In [11]:
#Read the CSV
ticket_data_df = pd.read_csv('../data-samples/customer_support_tickets.csv')
#This dataset has 8k+ rows, we shorten it to ~200 by randomly selecting some tickets
ticket_data_df = ticket_data_df.sample(n=200, random_state=42).reset_index(drop=True)
ticket_data_df = ticket_data_df.drop(columns=['Resolution']) #The resolution column is random gibberish, so we drop it
ticket_data_df

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,4831,James Smith,debra98@example.net,69,Female,Roomba Robot Vacuum,2020-03-24,Refund request,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,High,Phone,2023-06-01 12:59:31,NaN,NaN
1,7076,Erica Reed,carsonjames@example.net,64,Male,Roomba Robot Vacuum,2021-01-22,Product inquiry,Battery life,I'm having trouble connecting my {product_purc...,Closed,Low,Social media,2023-06-01 02:08:59,2023-06-01 10:49:59,4.0
2,4716,Mrs. Madison Thompson MD,taylorjames@example.com,41,Female,Philips Hue Lights,2020-06-07,Billing inquiry,Refund request,I'm having an issue with the {product_purchase...,Open,Low,Chat,NaN,NaN,NaN
3,2023,Sarah Nunez,martinezkenneth@example.org,62,Other,LG OLED,2021-02-20,Billing inquiry,Peripheral compatibility,I'm having an issue with the {product_purchase...,Closed,High,Social media,2023-06-01 13:54:11,2023-06-01 08:10:11,4.0
4,677,James White,rivasdavid@example.org,51,Female,Roomba Robot Vacuum,2021-08-01,Refund request,Peripheral compatibility,I'm having an issue with the {product_purchase...,Open,Medium,Social media,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1384,Shane James,mhoward@example.com,31,Male,Nikon D,2020-11-03,Technical issue,Account access,I'm having trouble connecting my {product_purc...,Closed,Low,Social media,2023-06-01 00:34:22,2023-06-01 01:31:22,2.0
196,8229,Shawn Knight,sanchezjohn@example.org,25,Male,Roomba Robot Vacuum,2021-08-22,Billing inquiry,Product setup,"My {product_purchased} crashed, and I lost all...",Closed,Critical,Email,2023-06-01 21:58:58,2023-06-02 00:12:58,2.0
197,6074,Nicole Ward,jnichols@example.net,49,Female,Canon DSLR Camera,2021-09-25,Refund request,Product compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,High,Email,2023-06-01 02:07:06,NaN,NaN
198,3780,Benjamin Noble,wgallegos@example.net,65,Other,Sony 4K HDR TV,2021-10-03,Technical issue,Refund request,I'm having an issue with the {product_purchase...,Open,Low,Email,NaN,NaN,NaN


In [12]:
#Save to csv
ticket_data_df.to_csv('../data-samples/customer_support_tickets_clean.csv', index=False)

### 3. Invoice Data
Invoice Data will consist of:
- Sample Invoice records (.csv) from Kaggle which contains shopping data of customers

We first download the .csv from kaggle (https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset) and load it:

In [15]:
#Read the CSV
invoice_data_df = pd.read_csv('../data-samples/customer_shopping_data.csv')
#The csv has ~100K rows, we select ~ 500 for the purpose of our qna demo
invoice_data_df = invoice_data_df.sample(n=500, random_state=42).reset_index(drop=True)
invoice_data_df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I704758,C971218,Female,56,Food & Beverage,4,20.92,Cash,14/05/2021,Kanyon
1,I218404,C949846,Male,22,Food & Beverage,5,26.15,Cash,9/9/2022,Metrocity
2,I195516,C440398,Female,53,Food & Beverage,4,20.92,Debit Card,1/4/2021,Kanyon
3,I833735,C113971,Female,65,Food & Beverage,5,26.15,Credit Card,3/8/2022,Cevahir AVM
4,I144592,C529879,Female,51,Shoes,4,2400.68,Credit Card,15/04/2021,Metropol AVM
...,...,...,...,...,...,...,...,...,...,...
495,I136209,C143246,Male,33,Cosmetics,3,121.98,Cash,21/06/2022,Kanyon
496,I210581,C275082,Female,39,Clothing,1,300.08,Credit Card,2/9/2021,Metrocity
497,I959066,C989688,Female,61,Shoes,2,1200.34,Cash,1/10/2021,Cevahir AVM
498,I157954,C241461,Female,33,Shoes,5,3000.85,Credit Card,25/02/2021,Zorlu Center


In [16]:
#Save to csv
invoice_data_df.to_csv('../data-samples/customer_shopping_data_clean.csv', index=False)

### 4. Resume Data (ATS)
Resume Data will consist of:
- Sample resume data (.csv) download from Kaggle. It contains a field which contains the Resume in text form, which is unstructured data.

Resumes are again a type of unstructured data: We will select ~300 random resumes and process them to get key information, which can be used to more easily retrieve the resume of the applicant when the query is provided to the llm.

In [21]:
#Load the csv
resume_data_df = pd.read_csv('../data-samples/Resume.csv')
# resume_data_df = resume_data_df.drop(columns=['Resume_html']) #Drop the html column
resume_data_df = resume_data_df.sample(n=300, random_state=42).reset_index(drop=True)
resume_data_df

,ID,Resume_str,Resume_html,Category
0,99244405,Kpandipou Koffi Summary ...,"<div class=""RNA skn-rbn1 fontsize fontface vma...",TEACHER
1,17562754,DIRECTOR OF DIGITAL TRANSFORMATION ...,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA
2,30311725,SENIOR PROJECT MANAGER Professi...,"<div class=""fontsize fontface vmargins hmargin...",CONSTRUCTION
3,19007667,CHEF Summary Experienced ca...,"<div class=""fontsize fontface vmargins hmargin...",CHEF
4,11065180,OPERATIONS MANAGER Summary E...,"<div class=""fontsize fontface vmargins hmargin...",BANKING
...,...,...,...,...
295,81310245,BUSINESS DEVELOPMENT EXECUTIVE ...,"<div class=""fontsize fontface vmargins hmargin...",BUSINESS-DEVELOPMENT
296,37664296,LEAD UX/UI DESIGNER Executive P...,"<div class=""fontsize fontface vmargins hmargin...",DESIGNER
297,36434348,INFORMATION TECHNOLOGY MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",INFORMATION-TECHNOLOGY
298,23085604,"CHIEF SYSTEM ARCHITECT, SVP SYSTEM IN...","<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA


In [23]:
#Display one of the resumes
print(resume_data_df.iloc[0]['Resume_str'])

           Kpandipou    Koffi         Summary      Compassionate teaching professional delivering exemplary support and assistance to teachers and students. Display exceptional Communication and problem solving skills.  Experience in office administration and public speaking. Attentive and adaptable, skilled in management of classroom operations. Effective in leveraging student feedback to create dynamic lesson plans that address individual strengths and weaknesses.  Dedicated and responsive team leader with proven skills in classroom management, behavior modification and individualized support.  Personable with experience using relationship-building to cultivate positive client, staff and management connections. Highly-developed communicator with outstanding skills in complex problem-solving and conflict resolution.  High-performing Administrative Assistant offering experience working with diverse client base and delivering exceptional results. Polished in managing client relations, a

In [ ]:
#TODO process each resume and extract key info